In [1]:
import sys
sys.path.append('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/')
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import yfinance as yf
import time
from datetime import timedelta, datetime
from systems.performance_reporter import StrategyReporter, PerformanceReporter
from systems.utils import MarketDataExtractor

In [2]:
market_data_extractor = MarketDataExtractor()
strategy_reporter = StrategyReporter()
performance_reporter = PerformanceReporter(market_data_extractor)

In [3]:
# All the Inputs for the Backtest that needs to be analyzed
testname = 'unknown_strategy_20250129_163354' # Profits not Reinvested
# testname = '3aa6bc1bc58ab86f2a23419d3737ec62991783e80d8d8802f5cc68a1cfb0a33b' # Profits Reinvested & Margin Deployed @ 0.65 of 3.3x
save_folder = "/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/data/yahoo/1d"
test_folder_path = f'/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/vault/backtest_reports/{testname}'
backtest_output = strategy_reporter.load_backtest_output(test_folder_path)
open_signals = backtest_output['open_signals']
closed_signals = backtest_output['closed_signals']
config_dict = backtest_output['config_dict']

In [6]:
signal_num = 0
open_signals[signal_num].orders[0].filled_price, closed_signals[signal_num].orders[1].filled_price

(43.92, 0.4389680294999999)

In [12]:
from pprint import pprint
pprint(open_signals[signal_num].model_dump_json(indent=2, exclude_unset=True))

('{\n'
 '  "strategy_name": "strategy_1",\n'
 '  "signal_id": "902e28f54ebf4f128d579a576c2bae89",\n'
 '  "timestamp": "2010-01-16T23:59:59Z",\n'
 '  "orders": [\n'
 '    {\n'
 '      "symbol": "JPM",\n'
 '      "orderQuantity": 10,\n'
 '      "orderDirection": "BUY",\n'
 '      "order_type": "MARKET",\n'
 '      "symbol_ltp": {\n'
 '        "2010-01-16T23:59:59Z": 43.92\n'
 '      },\n'
 '      "timeInForce": "DAY",\n'
 '      "status": "closed",\n'
 '      "filled_price": 43.92,\n'
 '      "filled_timestamp": "2010-01-16T23:59:59Z",\n'
 '      "order_id": "39b38a601cd3464f981f41eaa64e4d37",\n'
 '      "entryOrderBool": true,\n'
 '      "history": [\n'
 '        {\n'
 '          "symbol": "JPM",\n'
 '          "orderQuantity": 10,\n'
 '          "orderDirection": "BUY",\n'
 '          "order_type": "MARKET",\n'
 '          "symbol_ltp": {\n'
 '            "2010-01-16T23:59:59Z": 43.92\n'
 '          },\n'
 '          "timeInForce": "DAY",\n'
 '          "status": "pending",\n'
 '      

In [4]:
#Processing Open order to make them close, and remove history
open_orders_list = strategy_reporter.clean_up_order_list(open_orders, drop_history=True)
#Processing Closed orders to remove history
closed_orders_list = strategy_reporter.clean_up_order_list(closed_orders, drop_history=True)
#Merge both orders
final_list = closed_orders_list + open_orders_list
# Step 2: Sort the filtered orders by exit order `filled_timestamp`
sorted_orders = sorted(final_list, key=lambda x: x[1]['filled_timestamp'])
# Step 3: Fetch the first and last `filled_timestamp` with buffer
start_date = sorted_orders[0][1]['filled_timestamp'] - timedelta(days=365)
end_date = sorted_orders[-1][1]['filled_timestamp'] + timedelta(days=365)
# end_date = pd.Timestamp(datetime(2010, 1, 1))
# Current we are downloading csv file for below three index
indices = ["^DJI", "^IXIC", "^GSPC"]

index_data_dict = strategy_reporter.load_and_save_index_data(indices, start_date, end_date, save_folder)
rolling_window = int(max(60, (end_date - start_date).days / 20))
print({f"Analyzing with a rolling window of {rolling_window} days"})

NameError: name 'open_orders' is not defined

### This below function calculates profit/loss from the orders  **(exitPrice - entryPrice) * orderQuantity**

This function also plot the progress lines for 3 indexes provided earlier.

In [ ]:
starting_capital = 72267.75
trade_df_by_date = strategy_reporter.create_data_input_for_cumulative_returns_and_indices(final_list, index_data_dict, starting_capital)
strategy_reporter.plot_cumulative_returns_and_indices(trade_df_by_date, index_data_dict)

In [ ]:
sorted_orders[0]

# Plot Rolling charts

In [37]:
pair_trading_orders_dict = {}

for order_history in closed_orders:
    if order_history[0]['strategy_inputs']['strategy_order_id'] not in pair_trading_orders_dict.keys():
        pair_trading_orders_dict[order_history[0]['strategy_inputs']['strategy_order_id']] = []
    pair_trading_orders_dict[order_history[0]['strategy_inputs']['strategy_order_id']].append(
        {'symbol':order_history[0]['symbol'], 
         'direction':order_history[0]['orderDirection'],
         'entry_price':order_history[0]['fill_price'], 
         'exit_price':order_history[-1]['fill_price'], 
         'order_quantity':order_history[0]['orderQuantity'],
         'entry_timestamp':order_history[0]['filled_timestamp'], 
         'symbol_ltp-len':len(order_history[-1]['symbol_ltp']),
         'exit_timestamp':order_history[-1]['filled_timestamp'], 
         'order_value': (order_history[0]['fill_price']) * (1 if order_history[0]['orderDirection'] == 'BUY' else -1) * (order_history[0]['orderQuantity']),
         'profit': ((order_history[-1]['fill_price'] - order_history[0]['fill_price']) * (1 if order_history[0]['orderDirection'] == 'BUY' else -1) * (order_history[0]['orderQuantity']))
         })

In [38]:
pair_trading_orders_dict_ = {}
count = 0
for key, value in pair_trading_orders_dict.items():
    if len(value) == 2:
        # print(len(value))
        pair_trading_orders_dict_[key] = value
pair_trading_orders_dict = pair_trading_orders_dict_

In [ ]:
both_profit_count = 0
both_loss_count = 0
mixed_count = 0
profit_count = 0
loss_count = 0
profit_pct_list = []
loss_pct_list = []

for key, value in pair_trading_orders_dict.items():
    order_1, order_2 = value
    order_1_value = abs(order_1['order_value'])
    order_2_value = abs(order_2['order_value'])
    order_1_hold_dur = order_1['symbol_ltp-len']
    order_2_hold_dur = order_2['symbol_ltp-len']
    
    order_1_profit = order_1['profit']
    order_2_profit = order_2['profit']
    total_profit = order_1_profit + order_2_profit
    if order_1_profit > 0 and order_2_profit > 0:
        both_profit_count += 1
    elif order_1_profit < 0 and order_2_profit < 0:
        both_loss_count += 1
    else:
        mixed_count += 1
    
    if total_profit > 0:
        profit_count += 1
    else:
        loss_count += 1
    pnl = total_profit/(order_1_value + order_2_value) * 100
    
    if pnl > 0:
        profit_pct_list.append(pnl)
    else:
        loss_pct_list.append(pnl)
    print(f"Metrics: order_1: {round(order_1_profit, 1)}, Order 2 PnL: {round(order_2_profit, 1)}, Total PnL: {round(total_profit, 1)}, Total Order Value: {round(order_1_value + order_2_value, 2)}, PnL %: {round(pnl, 1)}%, Order Hold Duration: {(order_1_hold_dur, order_2_hold_dur)}")

In [ ]:
from numpy import average
average(profit_pct_list), average(loss_pct_list), len(profit_pct_list), len(loss_pct_list)

In [ ]:
average(profit_pct_list)/average(loss_pct_list)

In [ ]:
both_profit_count, both_loss_count, mixed_count, profit_count, loss_count

In [ ]:
profit_count / (profit_count + loss_count)

In [ ]:
pair_trading_orders_dict

In [ ]:
closed_orders[0]